In [41]:
import pandas as pd

In [42]:
df = pd.read_csv('../batchs/restaurant_1_week_002.csv')

In [43]:
df.head()

,Order Number,Order Date,Item Name,Quantity,Product Price,Total products
0,1388,2015-01-10 20:12:00,Onion Chutney,1,0.50,6
1,1388,2015-01-10 20:12:00,Mint Sauce,1,0.50,6
2,1388,2015-01-10 20:12:00,Lime Pickle,1,0.50,6
3,1388,2015-01-10 20:12:00,Paratha,1,2.95,6
4,1387,2015-01-10 16:55:00,Onion Bhaji,1,3.95,1


In [44]:
import glob
import os

# merging the files
files_joined = os.path.join('../batchs', "restaurant*.csv")

# Return a list of all joined files
list_files = glob.glob(files_joined)

# Merge files by joining all files
df2 = pd.concat(map(pd.read_csv, list_files), ignore_index=True)
print(df2)

        Order Number           Order Date                   Item Name  \
0             1388.0  2015-01-10 20:12:00               Onion Chutney   
1             1388.0  2015-01-10 20:12:00                  Mint Sauce   
2             1388.0  2015-01-10 20:12:00                 Lime Pickle   
3             1388.0  2015-01-10 20:12:00                     Paratha   
4             1387.0  2015-01-10 16:55:00                 Onion Bhaji   
...              ...                  ...                         ...   
193996           NaN  2019-12-07 20:57:00  Tandoori King Prawn Garlic   
193997           NaN  2019-12-07 17:33:00  Tandoori King Prawn Masala   
193998           NaN  2019-12-05 17:15:00  Tandoori King Prawn Masala   
193999           NaN  2019-12-02 18:00:00  Tandoori King Prawn Garlic   
194000           NaN  2019-12-06 02:59:00       House white wine 75cl   

        Quantity  Product Price  Total products  Order ID  
0              1           0.50               6       NaN  
1  

In [54]:
#df filtré sur des semaines souhaitées
start_week = 61
end_week = 70

def extract(data_dir: str, prefix: str, start_week: int, end_week: int) -> pd.DataFrame :
    df3 = pd.DataFrame()
    #data_dir = "../batchs"
    for i in range(start_week, end_week+1):
        file_path = os.path.join(data_dir,f'{prefix}_week_{i}.csv')
        if os.path.isfile(file_path):
            batch = pd.read_csv(file_path)
            df3 = pd.concat([df3,batch], sort=True)
    return df3

In [55]:
df3 = extract("C:/Users/104658/Vasseur_ML/batchs", "restaurant_1", 122, 126)
df3.head(5)

,Item Name,Order Date,Order Number,Product Price,Quantity,Total products
0,Mango Chutney,2017-05-07 17:56:00,5938,0.5,1,13
1,Onion Chutney,2017-05-07 17:56:00,5938,0.5,1,13
2,Mango Chutney,2017-05-05 19:25:00,5331,0.5,1,5
3,Mango Chutney,2017-05-05 19:01:00,5327,0.5,2,6
4,Mango Chutney,2017-05-05 17:16:00,5321,0.5,1,6


In [56]:
import numpy as np
# Nettoyage des données
def clean(df3: pd.DataFrame) -> pd.DataFrame:
    df3.columns = df3.columns.str.lower().str.replace(' ','_')
    df3['order_date'] = pd.to_datetime(df3['order_date'])
    # CA
    df3['total_product_price'] = df3['quantity']*df3['product_price']
    df3['cash_in'] =  df3.groupby('order_number')['total_product_price'].transform(np.sum)
    df3 = df3.drop(columns=['item_name','quantity','product_price','total_products','total_product_price'])
    df3 = df3.drop_duplicates()
    df3 = df3.reset_index(drop=True)
    return df3

In [58]:
df3.resample('1H', on='order_date').sum().reset_index()

,order_date,order_number,cash_in
0,2017-05-01 13:00:00,4195,20.40
1,2017-05-01 14:00:00,0,0.00
2,2017-05-01 15:00:00,0,0.00
3,2017-05-01 16:00:00,0,0.00
4,2017-05-01 17:00:00,8393,48.25
...,...,...,...
819,2017-06-04 16:00:00,0,0.00
820,2017-06-04 17:00:00,0,0.00
821,2017-06-04 18:00:00,9997,114.45
822,2017-06-04 19:00:00,10001,49.60


In [59]:
df2 = extract("C:/Users/104658/Vasseur_ML/batchs", "restaurant_2", 122, 126)
df2.head(5)

,Item Name,Order Date,Order ID,Product Price,Quantity,Total products
0,Mango Chutney,2017-05-07 19:27:00,10515,0.5,2,8
1,Red Sauce,2017-05-07 19:27:00,10515,0.5,2,8
2,Mango Chutney,2017-05-07 19:17:00,10514,0.5,1,9
3,Onion Chutney,2017-05-07 12:51:00,10511,0.5,1,6
4,Mint Sauce,2017-05-07 12:51:00,10511,0.5,1,6


In [61]:
# Nettoyage des données
def clean(df2: pd.DataFrame) -> pd.DataFrame:
    df2 = df2.rename(columns={'order_id':'order_number'})
    df2.columns = df2.columns.str.lower().str.replace(' ','_')
    df2['order_date'] = pd.to_datetime(df3['order_date'])
    # CA
    df2['total_product_price'] = df2['quantity']*df2['product_price']
    df2['cash_in'] =  df3.groupby('order_id')['total_product_price'].transform(np.sum)
    df2 = df2.drop(columns=['item_name','quantity','product_price','total_products','total_product_price'])
    df2 = df2.drop_duplicates()
    df2 = df2.reset_index(drop=True)
    return df3

In [57]:
df2 = clean(df2)
df3 = clean(df3)

In [62]:
def merge(df2: pd.DataFrame, df3: pd.DataFrame)->pd.DataFrame:
    df4 = pd.concat([df2,df3], sort=True)
    df4 = df4.drop(columns='order_number')
    df4 = df4.sort_values('order_date')
    df4 = df4.reset_index(drop=True)
    return df4


In [63]:
df4 = merge(df2,df3)
df4 = df4.resample('1H', on='order_date').sum().reset_index()
df4.head()

,order_date,Order ID,Product Price,Quantity,Total products,cash_in
0,2017-05-01 13:00:00,0.0,0.0,0.0,0.0,20.40
1,2017-05-01 14:00:00,0.0,0.0,0.0,0.0,0.00
2,2017-05-01 15:00:00,0.0,0.0,0.0,0.0,0.00
3,2017-05-01 16:00:00,0.0,0.0,0.0,0.0,0.00
4,2017-05-01 17:00:00,0.0,0.0,0.0,0.0,48.25
